In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [5]:
#get source page
URL = 'https://www.scriptsource.org/cms/scripts/page.php?item_id=script_overview'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

In [23]:
#list of links
base_url = 'https://www.scriptsource.org/cms/scripts/'
all_script_urls = []
for node in soup.find_all(class_='Table_Field_1'):
  all_script_urls.append(base_url + node.find('a').get('href'))

current_script_urls = all_script_urls[3:152]

In [129]:
#csv fields
name = []
lang_type = []
family = []
direction = []
baseline = []
case = []
white_space = []
ISO_15924_code = []
status = []
diacritics = []
contextual_forms = []
complex_positioning = []
reordering = []
split_graphs = []
ligatures = []
OpenType_tag = []
unicode_range = []

In [130]:
#loop through all pages
for url in current_script_urls:

  URL = url
  page = requests.get(URL)
  soup = BeautifulSoup(page.content, 'html.parser')

  #append language name
  name.append(soup.find_all('title')[1].get_text())

  #get features
  features = soup.find(class_='scr_features').find_all('dd')
  feature_names = []
  for feature in features:
    feature_names.append(feature.get_text())

  #get unicode status
  try:
    unicode_link = base_url + soup.find('a', string=re.compile('Unicode Status')).get('href')
    page = requests.get(unicode_link)
    soup = BeautifulSoup(page.content, 'html.parser')

    unicode_ = soup.find(class_='dDataViewTable').find_all(class_='Table_Odd_Row')[1].get_text()
    unicode_range.append(unicode_)
  except:
    unicode_range.append('N/A')

  #append features
  lang_type.append(feature_names[0].strip())
  diacritics.append(feature_names[1])
  family.append(feature_names[2])
  contextual_forms.append(feature_names[3])
  direction.append(feature_names[4])
  complex_positioning.append(feature_names[5])
  baseline.append(feature_names[6])
  reordering.append(feature_names[7])
  case.append(feature_names[8])
  split_graphs.append(feature_names[9])
  white_space.append(feature_names[10])
  ligatures.append(feature_names[11])
  ISO_15924_code.append(feature_names[12])
  OpenType_tag.append(feature_names[13])
  status.append(feature_names[14])


In [131]:
import pandas as pd

data = {
  'Name':name,
  'Type':lang_type,
  'Family':family,
  'Direction':direction,
  'Baseline':baseline,
  'Case':case,
  'White Space':white_space,
  'ISO 15924 Code':ISO_15924_code,
  'Status':status,
  'Diacritics':diacritics,
  'Contextual Forms':contextual_forms,
  'Complex Positioning':complex_positioning,
  'Reordering':reordering,
  'Split Graphs':split_graphs,
  'Ligatures':ligatures,
  'OpenType Tag':OpenType_tag,
  'Unicode Range':unicode_range
}

df = pd.DataFrame(data)

In [132]:
df

,Name,Type,Family,Direction,Baseline,Case,White Space,ISO 15924 Code,Status,Diacritics,Contextual Forms,Complex Positioning,Reordering,Split Graphs,Ligatures,OpenType Tag,Unicode Range
0,Adlam,alphabet,African,RTL,bottom,yes,between words,Adlm / 166 (right-to-left alphabetic),Current,yes,yes,[unknown],no,no,none,adlm,1E900..1E95F
1,Hebrew,abjad,Middle Eastern,RTL bidirectional,bottom,no,between words,Hebr / 125 (right-to-left alphabetic),Current,yes,yes,no,[unknown],[unknown],required,hebr,0000..007F
2,Pahawh Hmong,syllabary,East Asian,LTR,bottom,no,[unspecified],Hmng / 450 (syllabic),Current,yes,yes,[unknown],yes,[unknown],[unspecified],hmng,16B00..16B8F
3,Afaka,syllabary,American,LTR,bottom,no,discretionary,Afak / 439 (syllabic),Current,[unknown],[unknown],[unknown],[unknown],[unknown],[unspecified],[none],N/A
4,Hiragana,syllabary,East Asian,vertical (RTL) and horizontal (LTR),centered,no,discretionary,Hira / 410 (syllabic),Current,yes,no,[unknown],no,[unknown],[unspecified],kana,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,Hanifi Rohingya,,Mainland Southeast Asian,RTL,bottom,no,[unspecified],Rohg / 167 (right-to-left alphabetic),Current,yes,[unknown],yes,[unknown],[unknown],[unspecified],rohg,10D00..10D3F
145,Oriya (Odia),abugida,Indic,LTR,hanging,no,between words,Orya / 327 (alphasyllabic),Current,yes,[unknown],[unknown],[unknown],yes,required,"orya, ory2",0B00..0B7F
146,Zou,,Mainland Southeast Asian,LTR,hanging,yes,discretionary,[none],Current,yes,[unknown],[unknown],[unknown],[unknown],required,[none],N/A
147,Hanunoo (Hanunóo),abugida,Insular Southeast Asian,vertical (LTR),vertical,no,none,Hano / 371 (alphasyllabic),Current,yes,yes,yes,[unknown],[unknown],required,hano,1720..173F


In [133]:
df.to_csv('scripts_info.csv')